In [1]:
# Section 2.1 – Load Robot Sensor Data (Single Source)

# This cell explicitly loads the robot telemetry CSV used in the
# Data Streaming Visualization Workshop.
# We intentionally avoid unrelated datasets (e.g., housing data)
# to keep the MLOps pipeline focused on robot predictive maintenance.

import pandas as pd
from pathlib import Path

# The notebook is located in <project_root>/notebooks
# Robot CSV is stored in <project_root>/data/raw
raw_dir = Path.cwd().parent / "data" / "raw"
csv_path = raw_dir / "RMBR4-2_export_test.csv"

print("Using robot CSV:", csv_path)
print("CSV exists:", csv_path.exists())

# Load robot data into a Pandas DataFrame
df = pd.read_csv(csv_path)

print("Robot data loaded successfully.")
print("DataFrame shape:", df.shape)

df.head()


Using robot CSV: d:\Desktop\8010\LinearRegressionArchitecture_Workshop\data\raw\RMBR4-2_export_test.csv
CSV exists: True
Robot data loaded successfully.
DataFrame shape: (39672, 16)


,Trait,Axis #1,Axis #2,Axis #3,Axis #4,Axis #5,Axis #6,Axis #7,Axis #8,Axis #9,Axis #10,Axis #11,Axis #12,Axis #13,Axis #14,Time
0,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-17T12:18:23.660Z
1,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-17T12:18:25.472Z
2,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-17T12:18:27.348Z
3,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-17T12:18:29.222Z
4,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-17T12:18:31.117Z


In [ ]:
# Section 2.2 – Preprocess Robot Sensor Data

# 1. Standardizes column names for consistency across modules
# 2. Renames the time column to a canonical name: recorded_at
# 3. Converts timestamps to datetime objects
# 4. Removes unused robot axes to match the workshop scope
#
# This ensures the dataset is ready for:
#  Database ingestion
#  Streaming simulation
#  MLOps pipelines (training, evaluation, alerting)

# 1. Standardize column names
#     lowercase
#     replace spaces with underscores
#     remove '#' characters
df.columns = (
    df.columns
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("#", "", regex=False)
)

# 2. Rename time column (if present) to a canonical name
if "time" in df.columns and "recorded_at" not in df.columns:
    df.rename(columns={"time": "recorded_at"}, inplace=True)

# 3. Convert timestamp column to datetime for time-based operations
df["recorded_at"] = pd.to_datetime(df["recorded_at"])

# 4. Drop unused axes (the workshop focuses on axis_1 to axis_8)
unused_axes = [
    "axis_9", "axis_10", "axis_11",
    "axis_12", "axis_13", "axis_14"
]

df = df.drop(columns=[col for col in unused_axes if col in df.columns])

# 5. Quick sanity check
print("Preprocessing completed successfully.")
print("Final columns:")
print(df.columns.tolist())

print("\nFinal DataFrame shape:", df.shape)

df.head()


Preprocessing completed successfully.
Final columns:
['trait', 'axis_1', 'axis_2', 'axis_3', 'axis_4', 'axis_5', 'axis_6', 'axis_7', 'axis_8', 'recorded_at']

Final DataFrame shape: (39672, 10)


,trait,axis_1,axis_2,axis_3,axis_4,axis_5,axis_6,axis_7,axis_8,recorded_at
0,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:23.660000+00:00
1,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:25.472000+00:00
2,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:27.348000+00:00
3,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:29.222000+00:00
4,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:31.117000+00:00


In [3]:
# Section 2.3 – Simulate Multiple Robots (Minimum 3)

# The workshop and assignment require data from at least three
# different robots.
#
# Instead of duplicating CSV files, we simulate multiple robots
# by assigning a robot_id column to the existing telemetry data.
#
# This approach:
# - Matches real-world MLOps data modeling
# - Keeps the pipeline scalable
# - Works cleanly with databases and streaming


import pandas as pd

# Safety check: ensure df exists
if "df" not in globals():
    raise RuntimeError("DataFrame df is not defined. Run Section 2.1 and 2.2 first.")

# Number of robots to simulate (minimum required by assignment)
NUM_ROBOTS = 3

# Split the dataset evenly into N parts
df_parts = []
chunk_size = len(df) // NUM_ROBOTS

for i in range(NUM_ROBOTS):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < NUM_ROBOTS - 1 else len(df)

    part = df.iloc[start:end].copy()
    part["robot_id"] = f"robot_{i + 1}"

    df_parts.append(part)

# Combine all robot data back into a single DataFrame
df_multi_robot = pd.concat(df_parts, ignore_index=True)

# Reorder columns to keep identifiers first
cols = ["robot_id"] + [c for c in df_multi_robot.columns if c != "robot_id"]
df_multi_robot = df_multi_robot[cols]

print("Multi-robot dataset created successfully.")
print("Robots included:", df_multi_robot["robot_id"].unique())
print("Final shape:", df_multi_robot.shape)

df_multi_robot.head()


Multi-robot dataset created successfully.
Robots included: <StringArray>
['robot_1', 'robot_2', 'robot_3']
Length: 3, dtype: str
Final shape: (39672, 11)


,robot_id,trait,axis_1,axis_2,axis_3,axis_4,axis_5,axis_6,axis_7,axis_8,recorded_at
0,robot_1,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:23.660000+00:00
1,robot_1,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:25.472000+00:00
2,robot_1,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:27.348000+00:00
3,robot_1,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:29.222000+00:00
4,robot_1,current,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-17 12:18:31.117000+00:00


### Section 2.4 – Upload Multi-Robot Telemetry to Neon (PostgreSQL)


In [ ]:
# Section 2.4 – Upload Multi-Robot Telemetry to Neon (PostgreSQL)

import os
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError

# 1. Read Neon connection string from environment variable
DB_URL = os.getenv("NEON_DATABASE_URL")

if DB_URL is None:
    raise RuntimeError("NEON_DATABASE_URL is not set")

print("Connecting to Neon database...")

# 2. Create SQLAlchemy engine (Neon-friendly settings)

engine = create_engine(
    DB_URL,
    pool_pre_ping=True,   # check connection before using
    pool_recycle=300      # avoid stale connections
)

# 3. Upload multi-robot telemetry DataFrame

TABLE_NAME = "robot_telemetry"

try:
    df_multi_robot.to_sql(
        name=TABLE_NAME,
        con=engine,
        if_exists="replace",   # reproducible, workshop-friendly
        index=False,
        method="multi"         # faster & more reliable for large inserts
    )

    print("✅ Multi-robot telemetry uploaded successfully!")
    print(f"Table name: {TABLE_NAME}")
    print(f"Rows uploaded: {len(df_multi_robot)}")

except OperationalError as e:
    print("❌ Database operation failed.")
    print("This is often caused by Neon cold start or a transient connection issue.")
    print("👉 Just re-run this cell once more.")
    raise e



Connecting to Neon database...
✅ Multi-robot telemetry uploaded successfully!
Table name: robot_telemetry
Rows uploaded: 39672


In [12]:
# Section 2.5 – Synthetic Failure Target (Minimal)

# The provided robot telemetry dataset does not contain
# explicit failure timestamps.
# To support supervised Linear Regression (as required by
# the assignment), we define a simple synthetic target
# that represents "time to failure" based on sensor behavior.


import numpy as np

# Work on a copy to preserve raw data
df_reg = df_multi_robot.copy()

# Select sensor features
axis_cols = [c for c in df_reg.columns if c.startswith("axis_")]

# Define a simple proxy signal:
# Higher average axis load => closer to failure
df_reg["avg_axis_load"] = df_reg[axis_cols].mean(axis=1)

# Normalize the signal
min_val = df_reg["avg_axis_load"].min()
max_val = df_reg["avg_axis_load"].max()

normalized_load = (
    (df_reg["avg_axis_load"] - min_val) /
    (max_val - min_val + 1e-6)
)

# Define synthetic time-to-failure (in days)
MAX_DAYS_TO_FAILURE = 30
df_reg["time_to_failure_days"] = (
    (1 - normalized_load) * MAX_DAYS_TO_FAILURE
).clip(lower=0)

print("Synthetic regression target created.")
df_reg[["robot_id", "avg_axis_load", "time_to_failure_days"]].head()


Synthetic regression target created.


,robot_id,avg_axis_load,time_to_failure_days
0,robot_1,0.0,30.0
1,robot_1,0.0,30.0
2,robot_1,0.0,30.0
3,robot_1,0.0,30.0
4,robot_1,0.0,30.0


In [ ]:
# Section 2.6 – Synthetic Failure Target (Minimal)

# The provided robot telemetry dataset does not contain
# explicit failure timestamps.
# To support supervised Linear Regression (as required by
# the assignment), we define a simple synthetic target
# that represents "time to failure" based on sensor behavior.


import numpy as np

# Work on a copy to preserve raw data
df_reg = df_multi_robot.copy()

# Select sensor features
axis_cols = [c for c in df_reg.columns if c.startswith("axis_")]

# Define a simple proxy signal:
# Higher average axis load => closer to failure
df_reg["avg_axis_load"] = df_reg[axis_cols].mean(axis=1)

# Normalize the signal
min_val = df_reg["avg_axis_load"].min()
max_val = df_reg["avg_axis_load"].max()

normalized_load = (
    (df_reg["avg_axis_load"] - min_val) /
    (max_val - min_val + 1e-6)
)

# Define synthetic time-to-failure (in days)
MAX_DAYS_TO_FAILURE = 30
df_reg["time_to_failure_days"] = (
    (1 - normalized_load) * MAX_DAYS_TO_FAILURE
).clip(lower=0)

print("Synthetic regression target created.")
df_reg[["robot_id", "avg_axis_load", "time_to_failure_days"]].head()


In [ ]:
# Section 2.7 – Linear Regression 
# This section applies a basic Linear Regression model,

# Use sensor data (independent variables) to predict
# the estimated time to failure (dependent variable).

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [15]:
# Independent variables (features)
X = df_reg[axis_cols]

# Dependent variable (target)
y = df_reg["time_to_failure_days"]

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


Feature matrix shape: (39672, 8)
Target vector shape: (39672,)


In [16]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print("Training samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])


Training samples: 31737
Test samples: 7935


In [17]:
# Create and train the Linear Regression model
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train, y_train)

print("Linear Regression model trained.")


Linear Regression model trained.


In [18]:
# Predict on the test set
y_pred = lin_reg_model.predict(X_test)


In [ ]:
# Section 2.8 – Model Evaluation

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("R² Score:", r2)


MSE: 9.959416959083858e-26
RMSE: 3.155854394468138e-13
R² Score: 1.0


In [21]:
# Section 2.9 – 2-Week Alert Logic (Workshop-Aligned)

# Use the trained Linear Regression model to issue a
# maintenance alert when the predicted time to failure
# is within the next 2 weeks (14 days), as required
# by the assignment.


# Alert threshold (2 weeks)
ALERT_THRESHOLD_DAYS = 14


In [22]:
#  Generate Predictions

# Use the trained model to predict time-to-failure
df_alert = df_reg.copy()
df_alert["predicted_time_to_failure_days"] = lin_reg_model.predict(
    df_alert[axis_cols]
)

df_alert[
    ["robot_id", "predicted_time_to_failure_days"]
].head()


,robot_id,predicted_time_to_failure_days
0,robot_1,30.0
1,robot_1,30.0
2,robot_1,30.0
3,robot_1,30.0
4,robot_1,30.0


In [23]:
# Apply 2-Week Alert Rule

# Trigger alert if predicted failure is within 14 days
df_alert["maintenance_alert"] = (
    df_alert["predicted_time_to_failure_days"]
    <= ALERT_THRESHOLD_DAYS
)

print("Total alerts triggered:", df_alert["maintenance_alert"].sum())


Total alerts triggered: 1332


In [24]:
# Latest Alert Status per Robot

# Simulate a real-time check by looking at the latest
# telemetry record for each robot
latest_alerts = (
    df_alert
    .sort_values("recorded_at")
    .groupby("robot_id")
    .tail(1)
)

latest_alerts[
    ["robot_id", "predicted_time_to_failure_days", "maintenance_alert"]
]


,robot_id,predicted_time_to_failure_days,maintenance_alert
13223,robot_1,30.0,False
26447,robot_2,30.0,False
39671,robot_3,30.0,False


In [25]:
# Section 2.10 – Prepare Experiment Metadata
# This cell collects all relevant information about
# the current experiment, including model type,
# evaluation metrics, and alert configuration.

from datetime import datetime

experiment_record = {
    "timestamp": datetime.utcnow().isoformat(),
    "model": "LinearRegression",
    "num_features": len(axis_cols),
    "mse": mse,
    "rmse": rmse,
    "r2_score": r2,
    "alert_threshold_days": ALERT_THRESHOLD_DAYS
}

experiment_record


C:\Users\10122\AppData\Local\Temp\ipykernel_39188\2281915720.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


{'timestamp': '2026-01-30T06:02:04.701364',
 'model': 'LinearRegression',
 'num_features': 8,
 'mse': 9.959416959083858e-26,
 'rmse': np.float64(3.155854394468138e-13),
 'r2_score': 1.0,
 'alert_threshold_days': 14}

In [29]:
# Section 2.11 – Save Experiment Results
# Experiment results are persisted to a CSV file.

from pathlib import Path
import pandas as pd
from datetime import datetime

# Determine the project root directory
# (the parent directory of the "notebooks/" folder)
project_root = Path.cwd().parent

# Create (or reuse) the required "experiments/" directory
experiments_dir = project_root / "experiments"
experiments_dir.mkdir(exist_ok=True)

# Define the final experiment results file path
results_path = experiments_dir / "results.csv"

# Collect metadata and evaluation metrics for this experiment
experiment_record = {
    "timestamp": datetime.utcnow().isoformat(),
    "model": "LinearRegression",
    "num_features": len(axis_cols),
    "mse": mse,
    "rmse": rmse,
    "r2_score": r2,
    "alert_threshold_days": ALERT_THRESHOLD_DAYS
}

# Convert the experiment record to a DataFrame
results_df = pd.DataFrame([experiment_record])

# Persist results:
# - If results.csv already exists, append a new row
# - Otherwise, create the file with a header
if results_path.exists():
    results_df.to_csv(results_path, mode="a", header=False, index=False)
else:
    results_df.to_csv(results_path, index=False)

# Confirm where the results were saved
print("Final experiment results written to:")
print(results_path.resolve())

results_df


Final experiment results written to:
D:\Desktop\8010\LinearRegressionArchitecture_Workshop\experiments\results.csv


C:\Users\10122\AppData\Local\Temp\ipykernel_39188\2577394408.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


,timestamp,model,num_features,mse,rmse,r2_score,alert_threshold_days
0,2026-01-30T06:14:01.926434,LinearRegression,8,9.959417e-26,3.155854e-13,1.0,14


In [ ]:
# Section 2.5 – Synthetic Failure Target (Minimal)

# The provided robot telemetry dataset does not contain
# explicit failure timestamps.
# To support supervised Linear Regression (as required by
# the assignment), we define a simple synthetic target
# that represents "time to failure" based on sensor behavior.


import numpy as np

# Work on a copy to preserve raw data
df_reg = df_multi_robot.copy()

# Select sensor features
axis_cols = [c for c in df_reg.columns if c.startswith("axis_")]

# Define a simple proxy signal:
# Higher average axis load => closer to failure
df_reg["avg_axis_load"] = df_reg[axis_cols].mean(axis=1)

# Normalize the signal
min_val = df_reg["avg_axis_load"].min()
max_val = df_reg["avg_axis_load"].max()

normalized_load = (
    (df_reg["avg_axis_load"] - min_val) /
    (max_val - min_val + 1e-6)
)

# Define synthetic time-to-failure (in days)
MAX_DAYS_TO_FAILURE = 30
df_reg["time_to_failure_days"] = (
    (1 - normalized_load) * MAX_DAYS_TO_FAILURE
).clip(lower=0)

print("Synthetic regression target created.")
df_reg[["robot_id", "avg_axis_load", "time_to_failure_days"]].head()


Synthetic regression target created.


,robot_id,avg_axis_load,time_to_failure_days
0,robot_1,0.0,30.0
1,robot_1,0.0,30.0
2,robot_1,0.0,30.0
3,robot_1,0.0,30.0
4,robot_1,0.0,30.0


In [ ]:
"""
Section 2 – Data Loading and Database Ingestion (Workshop-Aligned)

What this cell does:
1) Finds the project root robustly (so it works no matter where you launch the notebook).
2) Locates the raw CSV under data/raw (auto-detects the file).
3) Loads the CSV into a Pandas DataFrame (df) and applies minimal preprocessing.
4) Uploads df to Neon PostgreSQL (connection string read from env var NEON_DATABASE_URL).
"""

import os
from pathlib import Path

import pandas as pd
from sqlalchemy import create_engine

# 0) Helper: find project root (the folder that contains data/raw)

def find_project_root(start_dir: Path) -> Path:
    start_dir = start_dir.resolve()
    for p in [start_dir] + list(start_dir.parents):
        if (p / "data" / "raw").exists():
            return p
    raise FileNotFoundError(
        "Could not find project root containing data/raw. "
        "Make sure your repo has a data/raw folder."
    )

# 1) Resolve paths (robust)

NOTEBOOK_DIR = Path.cwd().resolve()
PROJECT_ROOT = find_project_root(NOTEBOOK_DIR)
RAW_DIR = PROJECT_ROOT / "data" / "raw"

print("Notebook dir :", NOTEBOOK_DIR)
print("Project root :", PROJECT_ROOT)
print("Raw dir      :", RAW_DIR)
print("Raw dir exists:", RAW_DIR.exists())

# List files to help debugging
raw_files = sorted([p.name for p in RAW_DIR.glob("*")])
print("Files in data/raw:", raw_files)

# 2) Auto-pick the CSV file
#    - Prefer files that start with "RMBR" (your robot dataset)
#    - Otherwise pick the first CSV found

candidates = sorted(RAW_DIR.glob("RMBR*.csv"))
if not candidates:
    candidates = sorted(RAW_DIR.glob("*.csv"))

if not candidates:
    raise FileNotFoundError(
        "No CSV file found in data/raw. Please put your dataset CSV into data/raw/."
    )

CSV_PATH = candidates[0]
print("Using CSV:", CSV_PATH)

# 3) Load + minimal preprocessing

df = pd.read_csv(CSV_PATH)
print(f"CSV loaded. Shape: {df.shape}")

# Normalize column names
df.columns = (
    df.columns
      .astype(str)
      .str.lower()
      .str.strip()
      .str.replace(" ", "_")
      .str.replace("#", "", regex=False)
)

# Rename time column if present
if "time" in df.columns and "recorded_at" not in df.columns:
    df.rename(columns={"time": "recorded_at"}, inplace=True)

# Drop unused axes (optional)
cols_to_drop = ["axis_9", "axis_10", "axis_11", "axis_12", "axis_13", "axis_14"]



Notebook dir : D:\Desktop\8010\LinearRegressionArchitecture_Workshop\notebooks
Project root : D:\Desktop\8010\LinearRegressionArchitecture_Workshop
Raw dir      : D:\Desktop\8010\LinearRegressionArchitecture_Workshop\data\raw
Raw dir exists: True
Files in data/raw: ['RMBR4-2_export_test.csv', 'california_housing.csv']
Using CSV: D:\Desktop\8010\LinearRegressionArchitecture_Workshop\data\raw\RMBR4-2_export_test.csv
CSV loaded. Shape: (39672, 16)


In [8]:
from pathlib import Path
import pandas as pd

def find_project_root(start_dir: Path) -> Path:
    start_dir = start_dir.resolve()
    for p in [start_dir] + list(start_dir.parents):
        if (p / "data" / "raw").exists():
            return p
    raise FileNotFoundError("Could not find project root containing data/raw")

PROJECT_ROOT = find_project_root(Path.cwd())
CSV_PATH = PROJECT_ROOT / "data" / "raw" / "RMBR4-2_export_test.csv"

print("Project root:", PROJECT_ROOT)
print("Resolved CSV path:", CSV_PATH)
print("CSV exists:", CSV_PATH.exists())



Project root: D:\Desktop\8010\LinearRegressionArchitecture_Workshop
Resolved CSV path: D:\Desktop\8010\LinearRegressionArchitecture_Workshop\data\raw\RMBR4-2_export_test.csv
CSV exists: True


In [9]:
import pandas as pd
import time
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


class StreamingSimulator:
    """
    Simulates streaming robot telemetry data from a CSV file.

    Each call to nextDataPoint():
        1) Opens a database connection
        2) Inserts one record into a database table
        3) Refreshes the chart
        4) Closes the database connection
    """

    def __init__(self, csv_file, db_conn_str, table_name="robot_stream", delay=2, max_xticks=50):
        """
        Parameters:
        csv_file (str): Relative path to the CSV file
        db_conn_str (str): SQLAlchemy PostgreSQL connection string
        table_name (str): Database table name to insert streaming data
        delay (int/float): Seconds to wait between records (stream rate)
        max_xticks (int): Maximum number of time labels on the X-axis
        """
        # Load CSV into memory
        self.df = pd.read_csv(csv_file)

        # Standardize column names: lowercase + remove spaces + remove '#'
        self.df.columns = [
            col.lower().strip().replace(" ", "_").replace("#", "")
            for col in self.df.columns
        ]

        # Rename time column to recorded_at if needed
        if "time" in self.df.columns:
            self.df.rename(columns={"time": "recorded_at"}, inplace=True)

        # Convert timestamp column to datetime for better plotting
        self.df["recorded_at"] = pd.to_datetime(self.df["recorded_at"])

        # Detect axis columns (axis_1 to axis_8 only)
        self.axis_cols = [
            col for col in self.df.columns
            if col.startswith("axis_") and col.split("_")[1].isdigit() and 1 <= int(col.split("_")[1]) <= 8
        ]
        # Sort numerically (axis_1, axis_2, ..., axis_8)
        self.axis_cols = sorted(self.axis_cols, key=lambda x: int(x.split("_")[1]))

        # Store configs
        self.db_conn_str = db_conn_str
        self.table_name = table_name
        self.delay = delay
        self.max_xticks = max_xticks
        self.current_index = 0

        # Create SQLAlchemy engine (connection opened/closed per record)
        self.engine = create_engine(self.db_conn_str)

        # Plot initialization
        plt.ion()
        self.fig, self.ax = plt.subplots(figsize=(12, 7))
        self.x_data = []
        self.y_data_dict = {col: [] for col in self.axis_cols}

        print(f"Loaded CSV: {csv_file}")
        print(f"Detected Y-axis columns: {self.axis_cols}")

    def nextDataPoint(self):
        """
        Loads one record from the CSV into a DataFrame row,
        inserts it into the database, and refreshes the chart.
        """
        # Stop condition
        if self.current_index >= len(self.df):
            print("All data points have been streamed.")
            return None

        # Read the next row as a DataFrame
        row = self.df.iloc[[self.current_index]]

        # 1) Open connection -> 2) Insert record -> 4) Close connection
        try:
            with self.engine.connect() as conn:
                row.to_sql(self.table_name, conn, if_exists="append", index=False)
        except Exception as e:
            print(f"Database insert failed at index {self.current_index}: {e}")

        # 3) Update plot
        ts = row["recorded_at"].values[0]
        self.x_data.append(ts)

        for col in self.axis_cols:
            self.y_data_dict[col].append(row[col].values[0])

        self.ax.clear()

        # Plot all axes
        for col in self.axis_cols:
            self.ax.plot(self.x_data, self.y_data_dict[col], label=col, linewidth=1)

        # Chart formatting
        self.ax.set_title(f"Streaming Robot Axis Data ({self.current_index + 1}/{len(self.df)})")
        self.ax.set_xlabel("recorded_at")
        self.ax.set_ylabel("Axis Values")

        # Format time display
        self.ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

        # Limit the number of X-axis time labels
        if len(self.x_data) > self.max_xticks:
            step = max(1, len(self.x_data) // self.max_xticks)
            self.ax.set_xticks(self.x_data[::step])

        # Legend outside the plot area
        self.ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), fontsize="small")

        plt.xticks(rotation=45)
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()

        # Refresh the figure
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        plt.pause(0.05)

        # Move to the next row and simulate streaming delay
        self.current_index += 1
        time.sleep(self.delay)

        return row
